# see if I can generate predictions using the queen predictor to see how they compare to the approach used in phlegm 

In [1]:
!pip install -q transformers
!pip install fair-esm

import pickle
import numpy as np
import pandas as pd
import sys
from esm import Alphabet, pretrained
import matplotlib.pyplot as plt
import joblib
import os
import os.path
import torch
import seaborn as sns
from sklearn import metrics
from datetime import date
import torch
import re
import requests
from tqdm.auto import tqdm
import sys
import esm
from Bio import SeqIO


from transformers import EsmTokenizer, EsmForSequenceClassification

! git clone https://github.com/Furman-Lab/QUEEN/

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


/home/grig0076/miniconda3/envs/phynteny_transformer2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


fatal: destination path 'QUEEN' already exists and is not an empty directory.


In [2]:
tokenizer = EsmTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")
model = EsmForSequenceClassification.from_pretrained("facebook/esm2_t6_8M_UR50D", problem_type="multi_label_classification")

Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()
model = model.to(device)
model = model.eval()

In [21]:
# Load ESM-2 model
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0-32): 33 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=660, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((1280,), eps=1

In [31]:
import torch
import pickle
from Bio import SeqIO
from tqdm import tqdm
import gc

def process_sequences_in_batches(fasta_file_path, model, batch_converter, alphabet, 
                               batch_size=32, output_file="ESM_embed.pkl"):
    """
    Process protein sequences in batches to generate ESM embeddings efficiently.
    
    Args:
        fasta_file_path: Path to the FASTA file
        model: ESM model
        batch_converter: ESM batch converter
        alphabet: ESM alphabet
        batch_size: Number of sequences to process at once
        output_file: Output pickle file path
    """
    
    # First pass: collect all sequences
    print("Loading sequences...")
    all_data = []
    for record in SeqIO.parse(fasta_file_path, "fasta"):
        all_data.append((record.id, str(record.seq)))
    
    print(f"Loaded {len(all_data)} sequences")
    
    # Process in batches
    sequence_representations = []
    
    # Use tqdm for progress tracking
    for i in tqdm(range(0, len(all_data), batch_size), desc="Processing batches"):
        batch_data = all_data[i:i + batch_size]
        
        try:
            # Convert batch
            batch_labels, batch_strs, batch_tokens = batch_converter(batch_data)
            batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
            
            # Process with model
            with torch.no_grad():
                results = model(batch_tokens, repr_layers=[33], return_contacts=False)
            
            token_representations = results["representations"][33]
            
            # Extract per-residue representations for this batch
            for j, tokens_len in enumerate(batch_lens):
                # Average pooling to get sequence-level representation
                seq_repr = token_representations[j, 1:tokens_len-1].mean(0)
                sequence_representations.append(seq_repr)
            
            # Clean up GPU memory after each batch
            del batch_tokens, results, token_representations
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            
        except RuntimeError as e:
            if "out of memory" in str(e):
                print(f"OOM error at batch starting at index {i}. Consider reducing batch_size.")
                print(f"Current batch size: {batch_size}, sequences in this batch: {len(batch_data)}")
                # Optionally, you could retry with a smaller batch size
                raise e
            else:
                raise e
        
        # Optional: save intermediate results every N batches
        if (i // batch_size + 1) % 10 == 0:
            print(f"Processed {len(sequence_representations)} sequences so far...")
    
    # Save all representations
    print(f"Saving {len(sequence_representations)} embeddings to {output_file}")
    with open(output_file, "wb") as f:
        pickle.dump(sequence_representations, f)
    
    print("Processing complete!")
    return sequence_representations

# Alternative: Memory-efficient streaming version
def process_sequences_streaming(fasta_file_path, model, batch_converter, alphabet, 
                              batch_size=32, output_file="ESM_embed.pkl"):
    """
    Memory-efficient streaming version that doesn't load all sequences into memory first.
    """
    
    sequence_representations = []
    batch_data = []
    
    print("Processing sequences in streaming mode...")
    
    with tqdm(desc="Processing sequences") as pbar:
        for record in SeqIO.parse(fasta_file_path, "fasta"):
            batch_data.append((record.id, str(record.seq)))
            
            # Process when batch is full
            if len(batch_data) == batch_size:
                sequence_representations.extend(
                    process_batch(batch_data, model, batch_converter, alphabet)
                )
                batch_data = []  # Reset batch
                pbar.update(batch_size)
        
        # Process remaining sequences
        if batch_data:
            sequence_representations.extend(
                process_batch(batch_data, model, batch_converter, alphabet)
            )
            pbar.update(len(batch_data))
    
    # Save results
    print(f"Saving {len(sequence_representations)} embeddings to {output_file}")
    with open(output_file, "wb") as f:
        pickle.dump(sequence_representations, f)
    
    return sequence_representations

def process_batch(batch_data, model, batch_converter, alphabet):
    """Helper function to process a single batch."""
    batch_labels, batch_strs, batch_tokens = batch_converter(batch_data)
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
    
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33], return_contacts=False)
    
    token_representations = results["representations"][33]
    batch_representations = []
    
    for i, tokens_len in enumerate(batch_lens):
        seq_repr = token_representations[i, 1:tokens_len-1].mean(0)
        batch_representations.append(seq_repr)
    
    # Clean up
    del batch_tokens, results, token_representations
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    return batch_representations

# Usage example:
if __name__ == "__main__":

    
    fasta_file_path = '/home/grig0076/scratch/phlegm/PHROGs/phrog_representative_structures/monomers/fasta_known_state.faa'
    
    # Choose batch size based on your GPU memory
    # Start with 32, reduce if you get OOM errors
    batch_size = 4
    
    # Use the regular batching approach
    embeddings = process_sequences_in_batches(
        fasta_file_path, model, batch_converter, alphabet, 
        batch_size=batch_size, output_file="ESM_embed.pkl"
    )

Loading sequences...
Loaded 597 sequences


Processing batches:   7%|▋         | 10/150 [07:25<1:45:57, 45.41s/it]

Processed 40 sequences so far...


Processing batches:  13%|█▎        | 20/150 [14:38<1:35:18, 43.99s/it]

Processed 80 sequences so far...


...

Processed 520 sequences so far...


Processing batches:  93%|█████████▎| 140/150 [1:46:27<07:27, 44.79s/it]

Processed 560 sequences so far...


Processing batches: 100%|██████████| 150/150 [1:53:52<00:00, 45.55s/it]

Processed 597 sequences so far...
Saving 597 embeddings to ESM_embed.pkl
Processing complete!


In [ ]:
# see if the embeddings object is the same as the sequence representations object 

In [34]:
# If not previous tab, we generate it here:
full_tab_for_embed = pd.DataFrame()
np_list = []
# Detach the tensors to obtain a numpy array
for i, ten in enumerate(embeddings):
  ten=ten.detach().numpy()
  np_list.append(ten)
full_tab_for_embed["esm_embeddings"] = pd.Series(np_list)

In [35]:
# load model
model_location = "QUEEN/QUEEN_MLPmodel_final.pkl"
with open(model_location, "rb") as f:
  QUEEN_model = pickle.load(f)

/home/grig0076/miniconda3/envs/phynteny_transformer2/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.0.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/grig0076/miniconda3/envs/phynteny_transformer2/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MLPClassifier from version 1.0.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [52]:
y_test = QUEEN_model.predict(full_tab_for_embed["esm_embeddings"].to_list())
y_probs = QUEEN_model.predict_proba(full_tab_for_embed["esm_embeddings"].to_list())
y_predict_probs = [np.max(i) for i in y_probs] 
inv_map = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 10, 9: 12, 10: 14, 11: 24}
y_test_transformed = np.array([inv_map[x] for x in y_test])
print("These are the predicted labels:")
print(y_test_transformed)

These are the predicted labels:
[ 3  2  1  1  2  2  2  1  8  2  2  2  1  1  2  1  1  2  2  2  2  2  1  2
  1  2  2  1  3  1  2  2  1  3  2 12  1  3  1  1  2  3  1  2  1  2  2 12
  1  2  1  2  1  2  1  2  1  4  1  2  2  4  2  3  4  2  1  1  2  3  1  1
  8  4  2  5  2  2 12  1  2  3  2  3  2  8  4  2  8  3 12  2  1  6  1  1
  2  2  4  1  1  2  8  8  2  1  2  1  3  1  2  4  6  2  2  4  2  1  4  3
  1  1  1  1  3  2  2  1  6  3  3  2 24  6  1  2  4  2  1  6  1 12  1  5
  1  2  2  1  2  2  2  6  1  1  1  2  1  3  1  1  2  1  1  6  2  3  1  2
  2 12  1  1  2  2  2  1  1  4  1  8  3  2  2  1  1  8  2  1 10  1  4  1
  1  8  2 12 12  8  5  3  3  4  2  6  4  1  1  1  4  1  1  1  1  3  1  2
  8  2  1  2  6  2  1  2  1  3  1  1  1  1  8  1  1  3  2  2  2  2  2  2
  2  1 12  1  1  1  6  2  2  2  2  2  2  2  1  2  1  2  2  2  3  2  3  8
  8  1  1  2  8  2  2  2  8  3  1  2 12  2  2  3  1  3  4  4  2  2  6  1
  2  2  1  2  3  3  2  4  2  1  2  1  3  3  1  6  1  3  2  2  8  1  2  4
  4  2  6  4  5 12 

In [65]:
# get the headers so that these can be mapped across 
# Read headers into a list
headers = [record.id for record in SeqIO.parse(fasta_file_path, "fasta")] 
queen_results = pd.DataFrame({'phrog': headers, 'num_subunits': y_test_transformed, 'probability': y_predict_probs})
queen_results.to_csv('queen_phrog_validation_set_predictions.tsv', sep = '\t')

0.8462631837770839